In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import os
import pickle
import re

In [2]:
# comments have already been downloaded (using curl)
# and are in html/post_pages/
# there are about 400,000 of them

# we want to get:
# game name
# game id if available
# post author name
# post author id
# post content
# post length
# timestamp
# anything else???

post_dir = 'html/post_pages'

In [3]:
def info_from_game_post_file(file):
    pid_l = re.findall(r'\d+', file)
    pid = pid_l[0]
    did = 'post-' + pid
    # print(did)
    with open (file, 'r') as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    dclass = "post_content"
    div = soup.find('div', id=did)
    dtag = soup.div
    sdiv = dtag.attrs
    print(sdiv)

In [4]:
print(info_from_game_post_file('html/post_pages/435.html'))

{'id': 'header_widget_61163', 'class': ['header_widget']}
None
